# Introduction

Suite à la publication de 340 notices d'entités LRM par le groupe Systèmes et Données https://zenodo.org/record/7358799 voici un ensemble de scripts pour visualiser un peu leur contenu

Les données diffusées en iso2709 ont été converties en MarcXML pour faciliter les traitements.

**Plan**
 * [Les données initiales](#Les-donn%C3%A9es-initiales)
 * [Fonctions d'extraction](#Fonctions-d'extraction)
   * [Zones de liens entre entités OEMI](#Zones-de-liens-entre-entit%C3%A9s-OEMI)
   * [Classes et fonctions](#Classes-et-fonctions)
 * [Initialisation des entités](#Initialisation-des-entit%C3%A9s)
 * [Les manifestations](#Les-manifestations)
   * [Liens dans les manifs](#Liens-dans-les-manifestations)
   * [Liste des manifs et liens aux expressions](#Liste-des-manifs-et-liens-aux-expressions)
 * [Les expressions](#Les-expressions)
   * [Liste des expressions et liens aux oeuvres](#Liste-des-expressions-et-liens-aux-oeuvres)
 * [Coquilles rencontrées](#Coquilles-rencontrées)

# Les données initiales

In [11]:
manifs_filename = "UMB_Manifestations.xml"
oeuvres_expressions_filename = "UMA_Oeuvres_Expressions.xml"
autres_entites_filename = "UMA_Autres_Entites_Liees.xml"
items_filename = "UMH_Items.xml"

In [35]:
import SRUextraction as sru # import du fichier https://github.com/Lully/bnf-sru/blob/master/SRUextraction.py
from lxml import etree
from string import ascii_lowercase
import re

ns = {"marc": "http://www.loc.gov/MARC21/slim"}

In [13]:
manifs_file = etree.parse(manifs_filename)
oeuvres_expressions_file = etree.parse(oeuvres_expressions_filename)
autres_entites_file = etree.parse(autres_entites_filename)
items_file = etree.parse(items_filename)

# Fonctions d'extraction

## Zones de liens entre entités OEMI

In [14]:
tags_manif2expressions = {"507": "point d'accès autorisé Titre (manifestation pointant vers expression)", 
                          "577": "point d'accès autorisé Auteur-Titre (manifestation pointant vers expression)"}

tags_expression2oeuvres = {"532": "Point d’accès autorisé – Titre (expression pointant vers oeuvre)",
                           "542": "Point d’accès autorisé – Auteur/titre (expression pointant vers oeuvre)"}

## Classes et fonctions

In [37]:
dic_id2type = {}

class Record:
    def __init__(self, xml_record, rectype):
        self.xml = xml_record
        self.init_type = rectype
        self.id = sru.record2fieldvalue(self.xml, "001")
        self.txt = sru.xml2seq(self.xml)
        self.type = get_type(xml_record, rectype)
        self.label = get_label(self)
        dic_id2type[self.id] = self.type
    
    def __repr__(self):
        return etree.tostring(self.xml).decode("utf-8")


class Manifestation(Record):
    def __init__(self, xml_record, rectype):
        super().__init__(xml_record, rectype)
        self.toExpressions = manif2expression(self.xml)


class Oeuvre(Record):
    def __init__(self, xml_record, rectype):
        super().__init__(xml_record, rectype)
    
class Expression(Record):
    def __init__(self, xml_record, rectype):
        super().__init__(xml_record, rectype)
        self.toOeuvres  = expression2oeuvre(self.xml)


def manif2expression(xml_record):
    dict_expressions = {}
    for tag in tags_manif2expressions:
        for field_occ in xml_record.xpath(f"*[@tag='{tag}']"):
            recordid = sru.field2subfield(field_occ, "3")
            accesspoint = accesspoint2label(field_occ)
            dict_expressions[recordid] = accesspoint
    return dict_expressions

def expression2oeuvre(xml_record):
    dict_oeuvres = {}
    for tag in tags_expression2oeuvres:
        for field_occ in xml_record.xpath(f"*[@tag='{tag}']"):
            recordid = sru.field2subfield(field_occ, "3")
            accesspoint = accesspoint2label(field_occ)
            dict_oeuvres[recordid] = accesspoint
    return dict_oeuvres


    
def accesspoint2label(field):
    # Construction du point d'accès sans $
    label = []
    for subf in field.xpath("*[@code]"):
        code = subf.get("code")
        if code in ascii_lowercase and subf.text:
            label.append(subf.text)
    label = ", ".join(label)
    return label
    
def get_type(xml_record, rectype):
    entity_type = None
    if rectype in "mi":
        entity_type = rectype
    elif rectype in "oe":
        equiv = {"a": "o", "b": "e", "x": "x"}
        f154a = sru.record2fieldvalue(xml_record, "154$a")
        if f154a and f154a[1] in equiv:
            entity_type = equiv[f154a[1]]
        else:
            entity_type = "z"
    else:
        entity_type = rectype
    if entity_type is None:
        print(etree.tostring(xml_record))
    return entity_type
            
        
def get_label(record):
    label = []
    if record.type in "mipc":
        label.append(sru.record2fieldvalue(record.xml, "200$a"))
    elif record.type in "eox":
        for field in record.xml.xpath("*[@tag]"):
            tag = field.get("tag")
            if tag.startswith("2"):
                if sru.field2subfield(field, "a"):
                    label.append(sru.field2subfield(field, "a"))
                else:
                    label.append("*"*20)
                    label.append("$a vide")
                    label.append(sru.field2value(field))
    return ", ".join(label)


# Relations entre entités

Liens : 531 $3

Point d'accès structuré: zone 511

Note (non exploitable informatiquement) 370

# Initialisation des entités

In [38]:
manifs = [Manifestation(manif, "m") for manif in manifs_file.xpath(".//marc:record", namespaces=ns)]
oeuvres_expr = [Record(oe, "oe") for oe in oeuvres_expressions_file.xpath(".//marc:record", namespaces=ns)] 
oeuvres = []
expressions = []
oeuvres_or_expressions  =[]
for entity in oeuvres_expr:
    if entity.type == "o":
        oeuvre = Oeuvre(entity.xml, entity.init_type)
        oeuvres.append(oeuvre)
        
    elif entity.type == "e":
        expression = Expression(entity.xml, entity.init_type)
        expressions.append(expression)
    else:
        oeuvres_or_expressions.append(entity)

# Les manifestations

Besoins : 
* pour une manifestation, récupérer les métas principales :
  * titre, point d'accès, auteur
  * liens aux entités Expressions (et oeuvre ?)
* pour une manif, récupérer la liste des expressions et la liste des oeuvres (lien direct)
  * créer des attributs à la manifs, en fournissant l'expression source pour chaque oeuvre identifié

## Liens dans les manifs

* 507 : point d'accès autorisé Titre vers l'expression
* 577 : point d'accès autorisé Auteur-Titre vers l'expression

Chaque manifestation a un attribut .toExpression qui est un dictionnaire :
 * clé : numéro de notice d'expression
 * valeur : point d'accès (sans les dollars)

## Liste des manifs et liens aux expressions

Chaque instance de l'entité Manifestation a un attribut ".toExpressions" qui liste sous forme de dictionnaire les liens aux oeuvres (clé : numéro de notice d'expression ; valeur : point d'accès de l'expression)

In [17]:
for el in manifs:
    print(el.id, el.label)
    for expression in el.toExpressions:
        print(" "*10, expression, el.toExpressions[expression])

UMLRM0003 Nosferatu
           ULRM0002 Nosferatu, eine Symphonie des Grauens, film, Version restaurée, 60mn, 1995, Français
UMLRM0006 Dracula
           UMLRM0005 Stoker, Bram (1847-1912), Dracula, Anglais, Texte noté
UMLRM0009  Nosferatu, fantôme de la nuit
           UMLRM0008 Nosferatu, Phantom der Nacht, film, français
UMLRM0012 "Nosferatu le vampire"
           UMLRM0011 Palma, Paola (1971-….), Vezyroglou, Dimitri (1970-….), "Nosferatu le vampire", Français, Texte noté
UMLRM0015 Jean-Claude Gallotta, Nosferatu, ballet de l'Opéra
            UMLRM0015 Jean-Claude Gallotta, Nosferatu, ballet de l'Opéra, programme, 2006-05, Texte noté. Image fixe
UMLRM0018 Dracula
           UMLRM0005 Stoker, Bram (1847-1912), Dracula, Anglais, Texte noté
UMLRM0021  Dracula
           UMLRM0020 Stoker, Bram (1847-1912), Dracula, Français, Molitor, Texte noté
UMLRM0024 Dracula
           UMLRM0020 Stoker, Bram (1847-1912), Dracula, Français, Molitor, Texte noté
UMLRM0027 Dracula
           UMLRM0020

# Les expressions

## Liste des expressions et liens aux oeuvres

Chaque instance de l'entité Expression a un attribut ".toOeuvres" qui liste sous forme de dictionnaire les liens aux oeuvres (clé : numéro de notice d'oeuvre ; valeur : point d'accès de l'oeuvre)

In [29]:
for el in expressions:
    print(el.id, el.label)
    for oeuvre in el.toOeuvres:
        print(" "*10, oeuvre, el.toOeuvres[oeuvre])

UMLRM0002 Nosferatu, eine Symphonie des Grauens
           UMLRM0175 Les grands classiques du cinéma allemand, RDM vidéo, Anglais, allemand, français ; sous-titrage en français, Image animée
UMLRM0005 Stoker, Bram (1847-1912)
           UMLRM0056 A pour traduction, Stoker, Bram (1847-1912), Dracula, Français, Paul-Margueritte, Texte noté
           UMLRM0020 A pour traduction, Stoker, Bram (1847-1912), Dracula, Français, Molitor, Texte noté 
           UMLRM0041 A pour traduction, Stoker, Bram (1847-1912), Dracula, Français, Sirgent, Texte noté
           UMLRM0050 A pour traduction, Stoker, Bram (1847-1912), Dracula, Français, Finné, Texte noté
           UMLRM0053 Abrégé en, Stoker, Bram (1847-1912), Dracula, Version abrégée, Chicheportiche, Français, Texte noté
UMLRM0008 Nosferatu, Phantom der Nacht
UMLRM0011 Palma, Paola (1971-….)¤Vezyroglou, Dimitri (1970-….)
UMLRM0014 Jean-Claude Gallotta, Nosferatu, ballet de l'Opéra
UMLRM0020 Stoker, Bram (1847-1912)
           UMLRM0005 Est un

# Oeuvres expressions

Liste des titres

## Distinguer oeuvres et expressions

Label pos.9 permet de distinguer oeuvres et expressions : valeurs h / t (auteur-titre, ou titre seul) 

Zone 154 $a pos1 : a = oeuvre ; b = expression ; x = ne s'applique pas

Point d'accès
* 231 pour les oeuvres
* 232 pour les expressions

In [19]:
for el in oeuvres_expr:
    print(el.id, el.label)

UMLRM0001 ********************, $a vide, $7 ba0yba0y $8 freger $t Nosferatu, eine Symphonie des Grauens $c film
UMLRM0002 Nosferatu, eine Symphonie des Grauens
UMLRM0004 Stoker, Bram (1847-1912)
UMLRM0005 Stoker, Bram (1847-1912)
UMLRM0007  Nosferatu, Phantom der Nacht
UMLRM0008 Nosferatu, Phantom der Nacht
UMLRM0010 Palma, Paola (1971-….)¤Vezyroglou, Dimitri (1970-….)
UMLRM0011 Palma, Paola (1971-….)¤Vezyroglou, Dimitri (1970-….)
UMLRM0013 Jean-Claude Gallotta, Nosferatu, ballet de l'Opéra
UMLRM0014 Jean-Claude Gallotta, Nosferatu, ballet de l'Opéra
UMLRM0020 Stoker, Bram (1847-1912)
UMLRM0031 Stoker, Bram (1847-1912)
UMLRM0032 Stoker, Bram (1847-1912)
UMLRM0034 Humphries, Tudor (1953-….)
UMLRM0035 Humphries, Tudor (1953-….)
UMLRM0037 Stoker, Bram (1847-1912)
UMLRM0038 Stoker, Bram (1847-1912)
UMLRM0041 Stoker, Bram (1847-1912)
UMLRM0047 Stoker, Bram (1847-1912)
UMLRM0050 Stoker, Bram (1847-1912)
UMLRM0053 Stoker, Bram (1847-1912)
UMLRM0056 Stoker, Bram (1847-1912)
UMLRM0059 Stoker, B

# Générer un graphe Gephi

Objectif : générer 2 fichiers TSV (format en colonnes, tabulé) afin d'alimenter Gephi et de calculer un graphe

Un fichier listera les noeuds : 
 * identifiant (n° de notice)
 * label (point d'accès)
 * type (manifestation/expression/oeuvre) (à voir la pertinence d'y mettre aussi les items)

Un second fichier listera les liens entre ces noeuds
 

## Liste des noeuds

In [32]:
liste_noeuds = [["Identifiant", "Label", "Type"]]

def add_in_nodes_list(corpus, liste):
    for entity in corpus:
        liste.append([entity.id, entity.label, entity.type])

add_in_nodes_list(manifs, liste_noeuds)
add_in_nodes_list(expressions, liste_noeuds)
add_in_nodes_list(oeuvres, liste_noeuds)

## Liste des liens

In [33]:
liste_liens = []
for manif in manifs:
    for expr in manif.toExpressions:
        liste_liens.append([manif.id, expr])
for expr in expressions:
    for oeuvre in expr.toOeuvres:
        liste_liens.append([expr.id, oeuvre])

In [40]:
for lien in liste_liens:
    for recordid in lien:
        if re.fullmatch(r"UMLRM\d\d\d\d", recordid) is None:
            print(dic_id2type[lien[0]], lien)

m ['UMLRM0003', 'ULRM0002']
m ['UMLRM0015', ' UMLRM0015']
m ['UMLRM0060 ', 'UMLRM0059']
m ['UMLRM0060 ', 'UMLRM0153']
m ['UMLRM0101', 'UMLRM100']
m ['UMLRM0104', 'UMLRM100']
m ['UMLRM0122 ', 'UMLRM0189']
m ['UMLRM0122 ', 'UMLRM0121']
m ['UMLRM0134', 'ULRM00133']
m ['UMLRM0190', '']
e ['UMLRM0050', ' MLRM0172']
e ['UMLRM0076', '']
e ['UMLRM0161', ' UMLRM0035']
e ['UMLRM0167', '']
e ['UMLRM0175', '']


## Génération des fichiers

In [34]:
with open("liste_noeuds.txt", "w", encoding="utf-8") as file:
    for node in liste_noeuds:
        file.write("\t".join(node) + "\n")
with open("liste_liens.txt", "w", encoding="utf-8") as file:
    for link in liste_liens:
        file.write("\t".join(link) + "\n")

# Coquilles rencontrées

https://semestriel.framapad.org/p/bugs_unimarclrm